## Task 3 - Feature Engineering 

This notebook is for Feature Engineering

## Data Dictionary

<img src = "detail.jpg">

## Summary

1. Drop columns that has high amounts of zeros and missing values (> 5% ot total)
2. Decided to drop forecast columns since not sure how accurate the values are in future
3. Decided not to use the other historical energy and power data since too many zeros

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime
from datetime import datetime, timedelta
import scipy.stats
import pandas_profiling
from pandas_profiling import ProfileReport


%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

import warnings
warnings.filterwarnings('ignore')

#import feature_engine.missing_data_imputers as mdi
#from feature_engine.outlier_removers import Winsorizer
#from feature_engine import categorical_encoders as ce

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)

np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


In [2]:
df = pd.read_csv("train.csv", 
                 parse_dates=['date_activ', 'date_end', 'date_first_activ', 'date_modif_prod', 'date_renewal'],
                dayfirst=True)

In [3]:
df

,id,activity_new,campaign_disc_ele,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_first_activ,date_modif_prod,date_renewal,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,48ada52261e7cf58715202705a0451c9,esoiiifxdlbkcsluxmfuacbdckommixw,NaN,lmkebamcaaclubfxadlmueccxoimlema,309275,0,10025,2012-11-07,2016-11-06,NaT,2012-11-07,2015-11-09,NaN,NaN,NaN,NaN,26520.30,10025,0.0,359.29,0.095919,0.088347,58.995952,f,831.80,-41.76,-41.76,1,1732.36,3,ldkssxwpmemidmecebumciepifcamkci,180.000,0
1,24011ae4ebbe3035111d65fa7c15bc57,NaN,NaN,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,NaT,NaT,2015-06-23,NaN,NaN,NaN,NaN,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
2,d29c2c54acc38ff3c0614d0a653813dd,NaN,NaN,NaN,4660,0,0,2009-08-21,2016-08-30,NaT,2009-08-21,2015-08-31,NaN,NaN,NaN,NaN,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
3,764c75f661154dac3a6c254cd082ea7d,NaN,NaN,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,NaT,2010-04-16,2015-04-17,NaN,NaN,NaN,NaN,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0
4,bba03439a292a1e166f80264c16191cb,NaN,NaN,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,NaT,2010-03-30,2015-03-31,NaN,NaN,NaN,NaN,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16091,18463073fb097fc0ac5d3e040f356987,NaN,NaN,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,2012-05-24,2016-05-08,NaT,2015-05-08,2014-05-26,NaN,NaN,NaN,NaN,4648.01,0,0.0,18.57,0.138305,0.000000,44.311378,t,0.00,27.88,27.88,2,381.77,4,lxidpiddsbxsbosboudacockeimpuepw,15.000,0
16092,d0a6f71671571ed83b2645d23af6de00,NaN,NaN,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,2012-08-27,2016-08-27,2012-08-27,2012-08-27,2015-08-28,68.64,68.64,1254.65,15.94,631.69,181,0.0,144.03,0.100167,0.091892,58.995952,f,15.94,0.00,0.00,1,90.34,3,lxidpiddsbxsbosboudacockeimpuepw,6.000,1
16093,10e6828ddd62cbcf687cb74928c4c2d2,NaN,NaN,foosdfpfkusacimwkcsosbicdxkicaua,1844,0,179,2012-02-08,2016-02-07,NaT,2012-02-08,2015-02-09,NaN,NaN,NaN,NaN,190.39,179,0.0,129.60,0.116900,0.100015,40.606701,f,18.05,39.84,39.84,1,20.38,4,lxidpiddsbxsbosboudacockeimpuepw,15.935,1
16094,1cf20fd6206d7678d5bcafd28c53b4db,NaN,NaN,foosdfpfkusacimwkcsosbicdxkicaua,131,0,0,2012-08-30,2016-08-30,NaT,2012-08-30,2015-08-31,NaN,NaN,NaN,NaN,19.34,0,0.0,7.18,0.145711,0.000000,44.311378,f,0.00,13.08,13.08,1,0.96,3,lxidpiddsbxsbosboudacockeimpuepw,11.000,0


## Exploratory Data Analysis

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16096 entries, 0 to 16095
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        16096 non-null  object        
 1   activity_new              6551 non-null   object        
 2   campaign_disc_ele         0 non-null      float64       
 3   channel_sales             11878 non-null  object        
 4   cons_12m                  16096 non-null  int64         
 5   cons_gas_12m              16096 non-null  int64         
 6   cons_last_month           16096 non-null  int64         
 7   date_activ                16096 non-null  datetime64[ns]
 8   date_end                  16094 non-null  datetime64[ns]
 9   date_first_activ          3508 non-null   datetime64[ns]
 10  date_modif_prod           15939 non-null  datetime64[ns]
 11  date_renewal              16056 non-null  datetime64[ns]
 12  forecast_base_bill

In [5]:
df.describe()

,campaign_disc_ele,cons_12m,cons_gas_12m,cons_last_month,forecast_base_bill_ele,forecast_base_bill_year,forecast_bill_12m,forecast_cons,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_p1,forecast_price_energy_p2,forecast_price_pow_p1,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,churn
count,0.0,1.609600e+04,1.609600e+04,1.609600e+04,3508.000000,3508.000000,3508.000000,3508.000000,16096.000000,16096.000000,15970.000000,16096.000000,15970.000000,15970.000000,15970.000000,16096.000000,16083.000000,16083.000000,16096.000000,16081.000000,16096.000000,16093.000000,16096.000000
mean,NaN,1.948044e+05,3.191164e+04,1.946154e+04,335.843857,335.843857,3837.441866,206.845165,2370.555949,1907.347229,0.991547,70.309945,0.135901,0.052951,43.533496,196.123447,22.462276,21.460318,1.347788,217.987028,5.030629,20.604131,0.099093
std,NaN,6.795151e+05,1.775885e+05,8.235676e+04,649.406000,649.406000,5425.744327,455.634288,4035.085664,5257.364759,5.160969,79.023251,0.026252,0.048617,5.212252,494.366979,23.700883,27.917349,1.459808,366.742030,1.676101,21.772421,0.298796
min,NaN,-1.252760e+05,-3.037000e+03,-9.138600e+04,-364.940000,-364.940000,-2503.480000,0.000000,-16689.260000,-85627.000000,0.000000,-242.960000,0.000000,0.000000,-0.122184,-9038.210000,-525.540000,-615.660000,1.000000,-4148.990000,1.000000,1.000000,0.000000
25%,NaN,5.906250e+03,0.000000e+00,0.000000e+00,0.000000,0.000000,1158.175000,0.000000,513.230000,0.000000,0.000000,16.230000,0.115237,0.000000,40.606701,0.000000,11.960000,11.950000,1.000000,51.970000,4.000000,12.500000,0.000000
50%,NaN,1.533250e+04,0.000000e+00,9.010000e+02,162.955000,162.955000,2187.230000,42.215000,1179.160000,378.000000,0.000000,19.440000,0.142881,0.086163,44.311378,44.465000,21.090000,20.970000,1.000000,119.680000,5.000000,13.856000,0.000000
75%,NaN,5.022150e+04,0.000000e+00,4.127000e+03,396.185000,396.185000,4246.555000,228.117500,2692.077500,1994.250000,0.000000,131.470000,0.146348,0.098837,44.311378,218.090000,29.640000,29.640000,1.000000,275.810000,6.000000,19.800000,0.000000
max,NaN,1.609711e+07,4.188440e+06,4.538720e+06,12566.080000,12566.080000,81122.630000,9682.890000,103801.930000,175375.000000,50.000000,2411.690000,0.273963,0.195975,59.444710,15042.790000,374.640000,374.640000,32.000000,24570.650000,16.000000,500.000000,1.000000


In [6]:
df.columns

Index(['id', 'activity_new', 'campaign_disc_ele', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'date_activ', 'date_end', 'date_first_activ', 'date_modif_prod', 'date_renewal', 'forecast_base_bill_ele', 'forecast_base_bill_year', 'forecast_bill_12m', 'forecast_cons', 'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m', 'forecast_price_energy_p1', 'forecast_price_energy_p2', 'forecast_price_pow_p1', 'has_gas', 'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act', 'net_margin', 'num_years_antig', 'origin_up', 'pow_max', 'churn'], dtype='object')

### Drop unwanted features

In [7]:
df.columns

Index(['id', 'activity_new', 'campaign_disc_ele', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'date_activ', 'date_end', 'date_first_activ', 'date_modif_prod', 'date_renewal', 'forecast_base_bill_ele', 'forecast_base_bill_year', 'forecast_bill_12m', 'forecast_cons', 'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m', 'forecast_price_energy_p1', 'forecast_price_energy_p2', 'forecast_price_pow_p1', 'has_gas', 'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act', 'net_margin', 'num_years_antig', 'origin_up', 'pow_max', 'churn'], dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16096 entries, 0 to 16095
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        16096 non-null  object        
 1   activity_new              6551 non-null   object        
 2   campaign_disc_ele         0 non-null      float64       
 3   channel_sales             11878 non-null  object        
 4   cons_12m                  16096 non-null  int64         
 5   cons_gas_12m              16096 non-null  int64         
 6   cons_last_month           16096 non-null  int64         
 7   date_activ                16096 non-null  datetime64[ns]
 8   date_end                  16094 non-null  datetime64[ns]
 9   date_first_activ          3508 non-null   datetime64[ns]
 10  date_modif_prod           15939 non-null  datetime64[ns]
 11  date_renewal              16056 non-null  datetime64[ns]
 12  forecast_base_bill

In [9]:
df.drop(['activity_new', 'campaign_disc_ele', 'channel_sales','date_first_activ', 'cons_12m', 'cons_gas_12m',
         'date_modif_prod', 'date_renewal', 'forecast_base_bill_ele', 'forecast_base_bill_year', 
         'forecast_bill_12m', 'forecast_cons', 'forecast_cons_12m', 'forecast_cons_year', 
         'forecast_discount_energy','forecast_meter_rent_12m', 'forecast_price_energy_p1', 
         'forecast_price_energy_p2', 'forecast_price_pow_p1', 'imp_cons','origin_up'], axis=1, inplace=True)

In [10]:
df.head()

,id,cons_last_month,date_activ,date_end,has_gas,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,churn
0,48ada52261e7cf58715202705a0451c9,10025,2012-11-07,2016-11-06,f,-41.76,-41.76,1,1732.36,3,180.000,0
1,24011ae4ebbe3035111d65fa7c15bc57,0,2013-06-15,2016-06-15,t,25.44,25.44,2,678.99,3,43.648,1
2,d29c2c54acc38ff3c0614d0a653813dd,0,2009-08-21,2016-08-30,f,16.38,16.38,1,18.89,6,13.800,0
3,764c75f661154dac3a6c254cd082ea7d,0,2010-04-16,2016-04-16,f,28.60,28.60,1,6.60,6,13.856,0
4,bba03439a292a1e166f80264c16191cb,0,2010-03-30,2016-03-30,f,30.22,30.22,1,25.46,6,13.200,0


In [11]:
df["duration"] = (df.date_end - df.date_activ).dt.days

In [12]:
df.head()

,id,cons_last_month,date_activ,date_end,has_gas,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,churn,duration
0,48ada52261e7cf58715202705a0451c9,10025,2012-11-07,2016-11-06,f,-41.76,-41.76,1,1732.36,3,180.000,0,1460.0
1,24011ae4ebbe3035111d65fa7c15bc57,0,2013-06-15,2016-06-15,t,25.44,25.44,2,678.99,3,43.648,1,1096.0
2,d29c2c54acc38ff3c0614d0a653813dd,0,2009-08-21,2016-08-30,f,16.38,16.38,1,18.89,6,13.800,0,2566.0
3,764c75f661154dac3a6c254cd082ea7d,0,2010-04-16,2016-04-16,f,28.60,28.60,1,6.60,6,13.856,0,2192.0
4,bba03439a292a1e166f80264c16191cb,0,2010-03-30,2016-03-30,f,30.22,30.22,1,25.46,6,13.200,0,2192.0


In [13]:
df2 = pd.read_csv("customerchurn.csv")

In [14]:
df2

,id,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix,churn
0,0002203ffbb812588b632b9e628cc38d,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,0
1,0004351ebdd665e6ee664792efc4fd13,0.146427,0.000000,0.000000,44.385450,0.000000,0.000000,0
2,0010bcc39e42b3c2131ed2ce55246e3c,0.181559,0.000000,0.000000,45.319710,0.000000,0.000000,0
3,0010ee3855fdea87602a5b7aba8e42de,0.118757,0.098292,0.069032,40.647427,24.388455,16.258971,0
4,00114d74e963e47177db89bc70108537,0.147926,0.000000,0.000000,44.266930,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...
16091,ffef185810e44254c3a4c6395e6b4d8a,0.138863,0.115125,0.080780,40.896427,24.637456,16.507972,0
16092,fffac626da707b1b5ab11e8431a4d0a2,0.147137,0.000000,0.000000,44.311375,0.000000,0.000000,0
16093,fffc0cacd305dd51f316424bbb08d1bd,0.153879,0.129497,0.094842,41.160171,24.895768,16.763569,0
16094,fffe4f5646aa39c7f97f95ae2679ce64,0.123858,0.103499,0.073735,40.606699,24.364017,16.242678,0


In [15]:
df3 = pd.merge(left=df, right=df2, on="id", how="inner")

In [16]:
df3

,id,cons_last_month,date_activ,date_end,has_gas,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,churn_x,duration,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix,churn_y
0,48ada52261e7cf58715202705a0451c9,10025,2012-11-07,2016-11-06,f,-41.76,-41.76,1,1732.36,3,180.000,0,1460.0,0.103449,0.092115,0.067241,58.956502,36.356887,8.337051,0
1,24011ae4ebbe3035111d65fa7c15bc57,0,2013-06-15,2016-06-15,t,25.44,25.44,2,678.99,3,43.648,1,1096.0,0.122856,0.102137,0.072579,40.640023,24.384011,16.256008,1
2,d29c2c54acc38ff3c0614d0a653813dd,0,2009-08-21,2016-08-30,f,16.38,16.38,1,18.89,6,13.800,0,2566.0,0.149934,0.000000,0.000000,44.315416,0.000000,0.000000,0
3,764c75f661154dac3a6c254cd082ea7d,0,2010-04-16,2016-04-16,f,28.60,28.60,1,6.60,6,13.856,0,2192.0,0.170512,0.088421,0.000000,44.385450,0.000000,0.000000,0
4,bba03439a292a1e166f80264c16191cb,0,2010-03-30,2016-03-30,f,30.22,30.22,1,25.46,6,13.200,0,2192.0,0.151210,0.000000,0.000000,44.400265,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16091,18463073fb097fc0ac5d3e040f356987,0,2012-05-24,2016-05-08,t,27.88,27.88,2,381.77,4,15.000,0,1445.0,0.144124,0.000000,0.000000,44.370635,0.000000,0.000000,0
16092,d0a6f71671571ed83b2645d23af6de00,181,2012-08-27,2016-08-27,f,0.00,0.00,1,90.34,3,6.000,1,1461.0,0.106799,0.095406,0.070817,59.015674,36.393379,8.345418,1
16093,10e6828ddd62cbcf687cb74928c4c2d2,179,2012-02-08,2016-02-07,f,39.84,39.84,1,20.38,4,15.935,1,1460.0,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,1
16094,1cf20fd6206d7678d5bcafd28c53b4db,0,2012-08-30,2016-08-30,f,13.08,13.08,1,0.96,3,11.000,0,1461.0,0.149934,0.000000,0.000000,44.315416,0.000000,0.000000,0


In [17]:
df3.drop(['date_activ', 'date_end', 'id', 'churn_x'],axis=1,inplace=True)

In [18]:
df3.head()

,cons_last_month,has_gas,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,duration,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix,churn_y
0,10025,f,-41.76,-41.76,1,1732.36,3,180.000,1460.0,0.103449,0.092115,0.067241,58.956502,36.356887,8.337051,0
1,0,t,25.44,25.44,2,678.99,3,43.648,1096.0,0.122856,0.102137,0.072579,40.640023,24.384011,16.256008,1
2,0,f,16.38,16.38,1,18.89,6,13.800,2566.0,0.149934,0.000000,0.000000,44.315416,0.000000,0.000000,0
3,0,f,28.60,28.60,1,6.60,6,13.856,2192.0,0.170512,0.088421,0.000000,44.385450,0.000000,0.000000,0
4,0,f,30.22,30.22,1,25.46,6,13.200,2192.0,0.151210,0.000000,0.000000,44.400265,0.000000,0.000000,0


## Data Preprocessing

### Treat Missing Values

In [19]:
df3.isnull().sum()

cons_last_month          0
has_gas                  0
margin_gross_pow_ele    13
margin_net_pow_ele      13
nb_prod_act              0
net_margin              15
num_years_antig          0
pow_max                  3
duration                 2
price_p1_var             2
price_p2_var             2
price_p3_var             2
price_p1_fix             2
price_p2_fix             2
price_p3_fix             2
churn_y                  0
dtype: int64

In [20]:
df3.dropna(inplace=True)

In [21]:
df3.isnull().sum()

cons_last_month         0
has_gas                 0
margin_gross_pow_ele    0
margin_net_pow_ele      0
nb_prod_act             0
net_margin              0
num_years_antig         0
pow_max                 0
duration                0
price_p1_var            0
price_p2_var            0
price_p3_var            0
price_p1_fix            0
price_p2_fix            0
price_p3_fix            0
churn_y                 0
dtype: int64

### Treat Duplicate Values

In [22]:
df3.duplicated(keep='first').sum()

1

In [23]:
df3.drop_duplicates(ignore_index=True, inplace=True)

In [24]:
df3.duplicated(keep='first').sum()

0

### One-hot encoding

In [25]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16076 entries, 0 to 16075
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cons_last_month       16076 non-null  int64  
 1   has_gas               16076 non-null  object 
 2   margin_gross_pow_ele  16076 non-null  float64
 3   margin_net_pow_ele    16076 non-null  float64
 4   nb_prod_act           16076 non-null  int64  
 5   net_margin            16076 non-null  float64
 6   num_years_antig       16076 non-null  int64  
 7   pow_max               16076 non-null  float64
 8   duration              16076 non-null  float64
 9   price_p1_var          16076 non-null  float64
 10  price_p2_var          16076 non-null  float64
 11  price_p3_var          16076 non-null  float64
 12  price_p1_fix          16076 non-null  float64
 13  price_p2_fix          16076 non-null  float64
 14  price_p3_fix          16076 non-null  float64
 15  churn_y            

In [26]:
df3["has_gas"] = pd.get_dummies(data=df["has_gas"],drop_first=True)

In [27]:
df3

,cons_last_month,has_gas,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,duration,price_p1_var,price_p2_var,price_p3_var,price_p1_fix,price_p2_fix,price_p3_fix,churn_y
0,10025,0,-41.76,-41.76,1,1732.36,3,180.000,1460.0,0.103449,0.092115,0.067241,58.956502,36.356887,8.337051,0
1,0,1,25.44,25.44,2,678.99,3,43.648,1096.0,0.122856,0.102137,0.072579,40.640023,24.384011,16.256008,1
2,0,0,16.38,16.38,1,18.89,6,13.800,2566.0,0.149934,0.000000,0.000000,44.315416,0.000000,0.000000,0
3,0,0,28.60,28.60,1,6.60,6,13.856,2192.0,0.170512,0.088421,0.000000,44.385450,0.000000,0.000000,0
4,0,0,30.22,30.22,1,25.46,6,13.200,2192.0,0.151210,0.000000,0.000000,44.400265,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16071,0,0,27.88,27.88,2,381.77,4,15.000,1445.0,0.144124,0.000000,0.000000,44.370635,0.000000,0.000000,0
16072,181,0,0.00,0.00,1,90.34,3,6.000,1461.0,0.106799,0.095406,0.070817,59.015674,36.393379,8.345418,1
16073,179,0,39.84,39.84,1,20.38,4,15.935,1460.0,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,1
16074,0,0,13.08,13.08,1,0.96,3,11.000,1461.0,0.149934,0.000000,0.000000,44.315416,0.000000,0.000000,0


In [28]:
df3.rename(columns={"churn_y":"churn"}, inplace=True)

In [29]:
#df3.to_csv("final2.csv",index=False)